In [1]:
# !export PATH="${PATH}:/opt/gurobi912/linux64/bin"
# !export GUROBI_HOME="/home/victor-duraes/opt/gurobi912/linux64"
# !export LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:${GUROBI_HOME}/lib"

#Portfolio optimization all three energy sources

from __future__ import division
from pyomo.environ import *
import numpy as np
import time
from GetIdxInRadious import GetIdxOutRadious


start_time = time.time()
S_CPU_TIME=time.perf_counter()
######################

#LCOE range we want to investigate
#LCOE_Max=range(500,50,-10)

LCOE_Max=range(500,50,-1)
Nt=[200,200,200]# Total number of installed turbine units Wind/Wave/Ocean
#Upper bound for the number of installed turbines in a single cell Wind/Wave/Ocean
Radious=50
alpha=0.95

ResultsFileName='PortfolioOptimizationWindWaveOcean'+str(Nt[0])+"_"+str(Nt[1])+"_"+str(Nt[2])

In [2]:
Data=np.load("SyntheticData_CVAR_S_GAN_WithLoss.npz")
#Data=np.load("RealData_CVAR_S_GAN_WithLoss.npz")

#Wind
NumScenarios=365*30# 21900 #10950 #5475
WindEnergy=Data["WindEnergy"] #[pu]
WindEnergy=WindEnergy[0:NumScenarios,:]
WindLatLong=Data["WindLatLong"]
AnnualizedCostWind=Data["AverageAnnualCostPerWindTurbine"]*10**6 #[$/Year]
NumWindSites=WindEnergy.shape[1]
MaxNumOfWindTurbines=Data["MaxNumOfWindTurbines"]
RatedPowerWind=Data["RatedPowerWind"]
MaxNumOfWindTurbines[MaxNumOfWindTurbines>50]=50

#Wave
WaveEnergy=Data["WaveEnergy"] #[pu]
WaveEnergy=WaveEnergy[0:NumScenarios,:]
WaveLatLong=Data["WaveLatLong"]
NumWaveSites=WaveEnergy.shape[1]
RatedPowerWave=Data["RatedPowerWave"]
AnnualizedCostWave=Data["AverageAnnualCostPerWaveTurbine"]*10**6 #[$/Year-unit]
MaxNumOfWaveTurbines=Data["MaxNumOfWaveTurbines"]
MaxNumOfWaveTurbines[MaxNumOfWaveTurbines>50]=50

#Ocean
OceanEnergy=Data["OceanEnergy"] #[pu]
OceanEnergy=OceanEnergy[0:NumScenarios,:]
OceanLatLong=Data["OceanLatLong"]
AnnualizedCostOcean=Data["AverageAnnualCostPerOceanTurbine"]*10**6 #[$/Year]
NumOceanSites=OceanEnergy.shape[1]      
RatedPowerOcean=Data["RatedPowerOcean"]
MaxNumOfOceanTurbines=Data["MaxNumOfOceanTurbines"]
MaxNumOfOceanTurbines[MaxNumOfOceanTurbines>50]=50

RatedPowerSystem=(Nt[0]*RatedPowerWind+Nt[1]*RatedPowerWave+Nt[2]*RatedPowerOcean)*10**(-6)

#Vectorize maximum number of turbines per site location per technology
Nu=np.concatenate((MaxNumOfWindTurbines,MaxNumOfWaveTurbines,MaxNumOfOceanTurbines))

#Vectorize total number of installed turbines per technology
RHS_S=np.concatenate((np.full((NumWindSites), Nt[0]),np.full((NumWaveSites), Nt[1]),np.full((NumOceanSites), Nt[2])))

#Vectorize annualized cost for each site location and per technology
AnnCost=np.concatenate((AnnualizedCostWind,AnnualizedCostWave,AnnualizedCostOcean)) #Annualized cost [$/Year]

#Vectorize energy generation in each site location and energy resource
EnergyGeneration=np.concatenate((WindEnergy,WaveEnergy,OceanEnergy),axis=1)
EnergyGeneration_AVG=np.average(EnergyGeneration,axis=0)*8766#[MWh/Year] (8766 is the number of hours in a year)
NumSites=NumWindSites+NumWaveSites+NumOceanSites
NScenarios=OceanEnergy.shape[0]

In [3]:

MINLP = ConcreteModel()
MINLP.SiteWind = RangeSet(0,NumWindSites-1)
MINLP.SiteWave = RangeSet(NumWindSites,NumWindSites+NumWaveSites-1)
MINLP.SiteOcean = RangeSet(NumWindSites+NumWaveSites,NumSites-1)
MINLP.Site = RangeSet(0,NumSites-1)

MINLP.y = Var(MINLP.Site, domain=NonNegativeIntegers)# Integer variable to track the number of turbines used per site location
MINLP.s = Var(MINLP.Site, domain=Binary)# Integer variable to track the site location used for each technology

Sigma=np.cov(EnergyGeneration)

#Step to guarantee that the matrix get SDP
for i in range(Sigma.shape[0]):
    Sigma[i,i]=Sigma[i,i]+10**-8

def objective_rule(MINLP):   
    xtS=[sum(MINLP.y[i]*Sigma[i,j] for i in MINLP.Site) for j in MINLP.Site]

    return summation(xtS,MINLP.y)


MINLP.OBJ = Objective(rule = objective_rule, sense=minimize)

def NumTurbinesCell_rule(MINLP,i):
    return MINLP.y[i]<=Nu[i]

MINLP.Turbines_Cell = Constraint(MINLP.Site, rule=NumTurbinesCell_rule)

MINLP.TMaxTurbinesWind = Constraint(expr=sum(MINLP.y[i] for i in MINLP.SiteWind)==Nt[0])
MINLP.ChooseOneCircleWind1= Constraint(expr=sum(MINLP.s[i] for i in MINLP.SiteWind)==1)
    
MINLP.TMaxTurbinesWave = Constraint(expr=sum(MINLP.y[i] for i in MINLP.SiteWave)==Nt[1])
MINLP.ChooseOneCircleWave1= Constraint(expr=sum(MINLP.s[i] for i in MINLP.SiteWave)==1)

MINLP.TMaxTurbinesOcean = Constraint(expr=sum(MINLP.y[i] for i in MINLP.SiteOcean)==Nt[2])
MINLP.ChooseOneCircleOcean1= Constraint(expr=sum(MINLP.s[i] for i in MINLP.SiteOcean)==1)

IdxOut=GetIdxOutRadious(Radious, WindLatLong, WaveLatLong, OceanLatLong)

def MaximumRadious(MINLP,i):  
    
    return sum(MINLP.y[j] for j in np.where(IdxOut[i,:])[0])<=(1-MINLP.s[i])*RHS_S[i]        

MINLP.Maximum_Radious = Constraint(MINLP.Site, rule=MaximumRadious)

opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.01
#opt.options['max_iter'] = 500


In [4]:
SaveFeasibility, Save_LCOETarget, Save_S, Save_LCOE_Achieved = list(), list(), list(), list()
SaveYWind, SaveYWave, SaveYOcean,SaveCVaR = list(), list(), list(), list()
SaveYWind_LatLong, SaveYWave_LatLong, SaveYOcean_LatLong = list(), list(), list()
SaveVectorY=[]#Vectorized form. It helps computing new CVars

LowestLCOE=10**10
for LCOE_Idx in range(len(LCOE_Max)):
    LCOETarget=LCOE_Max[LCOE_Idx]
    
    if LCOETarget<LowestLCOE:    
        Bypass=0
        #upperbound for the LCOE
        MINLP.LCOE_Target = Constraint(expr=(sum(MINLP.y[i]*AnnCost[i] for i in MINLP.Site)-LCOETarget*sum(MINLP.y[i]*EnergyGeneration_AVG[i] for i in MINLP.Site)) <= 0)
       
        print("Running Model With LCOE= %.2f" % LCOETarget)
        
        try:
            results=opt.solve(MINLP, tee=True)
        except:
            Bypass=1
            MINLP.del_component(MINLP.LCOE_Target)  
    
        if Bypass==0:
            if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
                SaveFeasibility.append(1)
                Save_LCOETarget.append(LCOETarget)
                
                Optimal_Y=MINLP.y.get_values()
                Optimal_S=MINLP.s.get_values()
                
                Optimal_Y=np.reshape(np.array([Optimal_Y[i] for i in MINLP.Site]),(1,NumSites))
                Optimal_S=np.reshape(np.array([Optimal_S[i] for i in MINLP.Site]),(1,NumSites))
                
                Save_S.append(Optimal_S)
                SaveVectorY.append(Optimal_Y)
                
        
                CurrentLCOE=(np.sum(Optimal_Y*AnnCost)/np.sum(Optimal_Y*EnergyGeneration_AVG))
                Save_LCOE_Achieved.append(CurrentLCOE)
                SaveCVaR.append(-value(MINLP.OBJ))
                if LowestLCOE>CurrentLCOE:
                    LowestLCOE=CurrentLCOE
                    
                if Nt[0]!=0:
                    TempXWind=Optimal_Y[0, MINLP.SiteWind]
                    IdxWind=np.reshape(np.array([TempXWind> 10**-2]),-1)
                    SaveYWind.append(TempXWind[IdxWind])
                    SaveYWind_LatLong.append(WindLatLong[IdxWind,:])  
                    
                if Nt[1]!=0:  
                    TempXWave=Optimal_Y[0, MINLP.SiteWave]
                    IdxWave=np.reshape(np.array([TempXWave> 10**-2]),-1)
                    SaveYWave.append(TempXWave[IdxWave])
                    SaveYWave_LatLong.append(WaveLatLong[IdxWave,:])
        
                if Nt[2]!=0:        
                    TempXOcean=Optimal_Y[0, MINLP.SiteOcean]
                    IdxOcean=np.reshape(np.array([TempXOcean> 10**-2]),-1)
                    SaveYOcean.append(TempXOcean[IdxOcean])
                    SaveYOcean_LatLong.append(OceanLatLong[IdxOcean,:])
                
                #Delete constraint for its modification in the next step of the for loop
                MINLP.del_component(MINLP.LCOE_Target)
            
            else:# Something else is wrong
                MINLP.del_component(MINLP.LCOE_Target)  
                SaveVectorY.append(None) 
                SaveFeasibility.append(0)
                SaveCVaR.append(None)
                Save_LCOETarget.append(None)
                Save_S.append(None)
                Save_LCOE_Achieved.append(None)
                SaveYWind.append(None)                   
                SaveYWave.append(None)
                SaveYOcean.append(None)               
                SaveYWind_LatLong.append(None)
                SaveYWave_LatLong.append(None)                
                SaveYOcean_LatLong.append(None)
                break


Running Model With LCOE= 500.00
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 611 rows, 604 columns and 24536 nonzeros
Model fingerprint: 0xd2cf001e
Model has 45753 quadratic objective terms
Variable types: 0 continuous, 604 integer (302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+06]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 402834.33444
Presolve added 0 rows and 2 columns
Presolve removed 300 rows and 0 columns
Presolve time: 0.02s
Presolved: 311 rows, 606 columns, 23656 nonzeros
Presolved model has 45753 quadratic objective terms
Variable types: 0 continuous, 606 integer (302 binary)
Found heuristic solution: objective 368599.08231

Root relaxation: objective 4.572594e+03, 124 iterations, 0.00 s

     0     0 7378.67600    0   19 7488.13318 7378.67600  1.46%     -    0s
     0     0 7440.89282    0   10 7488.13318 7440.89282  0.63%     -    0s

Cutting planes:
  Implied bound: 5
  MIR: 2
  Relax-and-lift: 2

Explored 1 nodes (972 simplex iterations) in 0.63 seconds (0.38 work units)
Thread count was 32 (of 32 available processors)

Solution count 10: 7488.13 7488.13 7515.63 ... 12581

Optimal solution found (tolerance 1.00e-02)
Best objective 7.488133182428e+03, best bound 7.440892820756e+03, gap 0.6309%
Running Model With LCOE= 226.00
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 611 rows, 604 columns and 24536 nonzeros
Model fingerprint: 0x1556285c
Model has 45753 quadratic objective terms
Variable types: 0 continuous, 604 integer (302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [0e+00, 0e+00]

  Relax-and-lift: 11

Explored 1 nodes (812 simplex iterations) in 0.84 seconds (0.42 work units)
Thread count was 32 (of 32 available processors)

Solution count 10: 7561.09 7561.12 7561.12 ... 12859.9

Optimal solution found (tolerance 1.00e-02)
Best objective 7.561089560976e+03, best bound 7.523723564846e+03, gap 0.4942%
Running Model With LCOE= 224.00
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 611 rows, 604 columns and 24536 nonzeros
Model fingerprint: 0xc8983210
Model has 45753 quadratic objective terms
Variable types: 0 continuous, 604 integer (302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve added 0 rows and 2 columns
Presolve removed 300 rows and 0 columns
Presolve time

Thread count was 32 (of 32 available processors)

Solution count 10: 7806.99 7808.18 7812.56 ... 14711.4

Optimal solution found (tolerance 1.00e-02)
Best objective 7.806994595278e+03, best bound 7.736937358455e+03, gap 0.8974%
Running Model With LCOE= 222.00
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 611 rows, 604 columns and 24536 nonzeros
Model fingerprint: 0x44a92cec
Model has 45753 quadratic objective terms
Variable types: 0 continuous, 604 integer (302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve added 0 rows and 2 columns
Presolve removed 300 rows and 0 columns
Presolve time: 0.02s
Presolved: 311 rows, 606 columns, 23656 nonzeros
Presolved model has 45753 quadratic objec

     0     0 8082.20661    0   20 8380.31560 8082.20661  3.56%     -    0s
     0     0 8085.43794    0   14 8380.31560 8085.43794  3.52%     -    0s
     0     0 8085.43794    0   14 8380.31560 8085.43794  3.52%     -    0s
H    0     0                    8134.5076607 8085.43794  0.60%     -    0s

Cutting planes:
  Implied bound: 9
  MIR: 9
  StrongCG: 1
  Relax-and-lift: 4

Explored 1 nodes (845 simplex iterations) in 0.51 seconds (0.37 work units)
Thread count was 32 (of 32 available processors)

Solution count 10: 8134.51 8380.32 8409.78 ... 16566.2

Optimal solution found (tolerance 1.00e-02)
Best objective 8.134507660724e+03, best bound 8.085437941245e+03, gap 0.6032%
Running Model With LCOE= 220.00
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 611 rows, 604 columns and 24536 nonzeros
Model fingerprint: 0x25fae3fa
Model has 45753 quadra

...

Best objective 9.855478546925e+03, best bound 9.851836718710e+03, gap 0.0370%
Running Model With LCOE= 212.00
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 611 rows, 604 columns and 24536 nonzeros
Model fingerprint: 0xf8d8f6ce
Model has 45753 quadratic objective terms
Variable types: 0 continuous, 604 integer (302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve added 0 rows and 2 columns
Presolve removed 300 rows and 0 columns
Presolve time: 0.02s
Presolved: 311 rows, 606 columns, 23656 nonzeros
Presolved model has 45753 quadratic objective terms
Variable types: 0 continuous, 606 integer (302 binary)

Root relaxation: objective 7.632710e+03, 156 iterations, 0.01 seconds (0.00 work un

H    0     0                    32620.905957 8359.24150  74.4%     -    0s
H    0     0                    27282.966942 8880.31496  67.5%     -    0s
     0     0 8880.31496    0   21 27282.9669 8880.31496  67.5%     -    0s
H    0     0                    18623.964478 8880.31496  52.3%     -    0s
H    0     0                    11346.114189 8880.31496  21.7%     -    0s
     0     0 8880.31496    0   22 11346.1142 8880.31496  21.7%     -    0s
     0     0 9181.61392    0   22 11346.1142 9181.61392  19.1%     -    0s
     0     0 9331.06480    0   29 11346.1142 9331.06480  17.8%     -    0s
     0     0 10420.4163    0   20 11346.1142 10420.4163  8.16%     -    0s
     0     0 10504.3447    0   21 11346.1142 10504.3447  7.42%     -    0s
     0     0 10869.7263    0   25 11346.1142 10869.7263  4.20%     -    0s
     0     0 10869.7263    0   23 11346.1142 10869.7263  4.20%     -    0s
     0     0 10869.7263    0   17 11346.1142 10869.7263  4.20%     -    0s
H    0     0             

     0     0 12070.7748    0   32 13315.2030 12070.7748  9.35%     -    0s
H    0     0                    13314.877155 12070.7748  9.34%     -    0s
     0     0 12236.1855    0   32 13314.8772 12236.1855  8.10%     -    0s
     0     0 12287.8016    0   27 13314.8772 12287.8016  7.71%     -    0s
H    0     0                    13314.240065 12287.8016  7.71%     -    0s
     0     0 12797.7005    0   35 13314.2401 12797.7005  3.88%     -    0s
     0     0 12797.7005    0   18 13314.2401 12797.7005  3.88%     -    0s
H    0     0                    13313.242955 12797.7005  3.87%     -    0s
     0     0 12797.7005    0   28 13313.2430 12797.7005  3.87%     -    0s
     0     0 12797.7005    0   26 13313.2430 12797.7005  3.87%     -    0s
     0     0 12873.7439    0   22 13313.2430 12873.7439  3.30%     -    0s
     0     0 12885.0545    0   21 13313.2430 12885.0545  3.22%     -    0s
     0     0 12930.6228    0   22 13313.2430 12930.6228  2.87%     -    0s
     0     0 12945.4347  

H    0     0                    17310.177472 12881.0152  25.6%     -    0s
     0     0 12881.3135    0   19 17310.1775 12881.3135  25.6%     -    0s
     0     0 12881.3135    0   19 17310.1775 12881.3135  25.6%     -    0s
     0     0 15553.2944    0   30 17310.1775 15553.2944  10.1%     -    0s
     0     0 16298.9975    0   24 17310.1775 16298.9975  5.84%     -    0s
     0     0 16820.3227    0   23 17310.1775 16820.3227  2.83%     -    0s
     0     0 16820.3227    0   14 17310.1775 16820.3227  2.83%     -    0s
     0     0 16820.3227    0   23 17310.1775 16820.3227  2.83%     -    0s
     0     0 16820.3227    0   21 17310.1775 16820.3227  2.83%     -    0s
     0     0 16820.3227    0   23 17310.1775 16820.3227  2.83%     -    0s
     0     0 16982.6187    0   28 17310.1775 16982.6187  1.89%     -    0s
     0     0 17002.8986    0   22 17310.1775 17002.8986  1.78%     -    0s
     0     0 17002.8986    0   22 17310.1775 17002.8986  1.78%     -    0s
     0     0 17182.2735  


Optimal solution found (tolerance 1.00e-02)
Best objective 2.745280748912e+04, best bound 2.744596409866e+04, gap 0.0249%
Running Model With LCOE= 203.00
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 611 rows, 604 columns and 24536 nonzeros
Model fingerprint: 0xffecd555
Model has 45753 quadratic objective terms
Variable types: 0 continuous, 604 integer (302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve added 0 rows and 2 columns
Presolve removed 300 rows and 0 columns
Presolve time: 0.02s
Presolved: 311 rows, 606 columns, 23656 nonzeros
Presolved model has 45753 quadratic objective terms
Variable types: 0 continuous, 606 integer (302 binary)

Root relaxation: objective 2.591307e+0

In [5]:
np.savez("./P1_PO_30Y/Var"+ResultsFileName + ".npz", TotalNumTurbines=Nt,SaveFeasibility=SaveFeasibility,
         Save_LCOETarget=Save_LCOETarget, Save_S=Save_S, Save_LCOE_Achieved=Save_LCOE_Achieved, SaveYWind=SaveYWind,SaveYWave=SaveYWave,
         SaveYOcean=SaveYOcean, SaveYWind_LatLong=SaveYWind_LatLong, SaveYWave_LatLong=SaveYWave_LatLong, SaveYOcean_LatLong=SaveYOcean_LatLong,
         SaveCVaR=SaveCVaR,SaveVectorY=SaveVectorY)


C:\Users\Remote\anaconda3\envs\Gurobi\lib\site-packages\numpy\lib\npyio.py:696: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
